In [ ]:
#| default_exp project

In [ ]:
#| export
from __future__ import annotations

# Project
> Global console configuration for consistent output across projects

This module provides a singleton `rich.Console` instance configured for your entire project. Benefits:

- Consistent output — Single console configuration across all modules
- Easy access — Import and use without repeated configuration
- Theme support — Optional integration with `rich_theme_manager`
- Notebook-aware — Automatically adapts to Jupyter vs terminal

## Key Components

- `setup_console` — Initialize global console with width, theme, and options
- `get_console` — Access the global console instance from any module
- `get_theme` — Load themes from `rich_theme_manager` if available

## Usage Pattern

**In your project's main module or notebook:**
```python
from pote.project import setup_console

console, cprint = setup_console(width=120, theme='monokai')
```

**In other modules:**
```python
from pote.project import get_console

console = get_console()
console.print("[bold]Hello[/bold]")
```

## Why a Global Console?

Rich's `Console` maintains state (width, theme, output buffer). Creating multiple instances can lead to inconsistent output. A global console ensures:
- Consistent formatting across your codebase
- Single theme configuration
- Predictable line wrapping
- Shared output context

<!-- # Prologue -->

In [ ]:
#| export
import pathlib
from typing import Callable

import fastcore.all as FC
from rich.console import Console


# console

## Theme Customization

`setup_console` supports theme customization through:

1. **String theme name**: Loads from `rich_theme_manager` if installed
2. **Theme object**: Pass a `rich.theme.Theme` instance directly
3. **None**: Uses default Rich theme

**Example with theme manager:**
```python
# Requires: pip install rich-theme-manager
console, cprint = setup_console(width=120, theme='monokai')
```

**Example with custom theme:**
```python
from rich.theme import Theme

custom_theme = Theme({
    "info": "cyan",
    "warning": "yellow",
    "error": "bold red"
})
console, cprint = setup_console(theme=custom_theme)
```

In [ ]:
#| exporti

# _console = Console(width=107, tab_size= 4, theme=dark_theme, force_jupyter=True if FC.IN_NOTEBOOK else None)
_CONSOLE: Console = None  # type: ignore

def get_theme(theme:str | object | None):
    if isinstance(theme, str):
        try:
            from rich_theme_manager import ThemeManager  # type: ignore
            theme_dir = pathlib.Path("~/.rich_theme_manager/themes").expanduser()
            theme_manager = ThemeManager(theme_dir=theme_dir.absolute().as_posix())
            try: theme = theme_manager.get(theme)
            except ValueError: theme = None
        except ImportError:
            theme = None
    # return theme or Theme('default', styles={"repr.tag_contents": "white"})
    return theme


In [ ]:
#| export

def get_console() -> Console:
    "Globally configured console"
    global _CONSOLE
    return _CONSOLE

def setup_console(
        width: int = 140,
        theme: str | object | None = None,
        clear=False,
        rule=None
) -> tuple[Console, Callable[..., None]]:
    global _CONSOLE
    theme = get_theme(theme)
    params = dict(width=width, tab_size=4, theme=theme, force_jupyter=True if FC.IN_IPYTHON else None)
    _CONSOLE = Console(**params)  # type: ignore
    if clear: _CONSOLE.clear()
    if rule:_CONSOLE.rule(rule)
    return _CONSOLE, _CONSOLE.print

setup_console();

In [ ]:
print(_CONSOLE)

<console width=140 ColorSystem.TRUECOLOR>


----
<!-- # Colophon -->

In [ ]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '00_project.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
